In [66]:
import astropy.units as u
from astroquery.xmatch import XMatch
from astropy.table import Column,QTable
from astroquery.vizier import Vizier
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
import matplotlib.pyplot as plt
from astroquery.xmatch import XMatch
import numpy as np
import pandas as pd
from astropy.io import ascii

coord = SkyCoord("04:30:47.99", "+64:50:56.5", unit=(u.hourangle, u.deg), frame='icrs')
rad = 6 * u.arcminute

catalogs = ['vizier:I/350/gaiaedr3', 'vizier:II/246/out', 'vizier:II/328/allwise', 'vizier:II/349/ps1']
catalog = ['I/350/gaiaedr3', 'II/246/out', 'II/328/allwise', 'II/349/ps1']

data = QTable()
df_gaia = QTable(names=('RA_ICRS','DE_ICRS'))
data_2mass = QTable()
data_wise = QTable()
data_panstarrs = QTable()

df_gaia = QTable(names=('RA_ICRS', 'DE_ICRS', 'Gmag'))
df_2mass = QTable(names=('RAJ2000', 'DEJ2000', 'Jmag', 'Hmag', 'Kmag'))
df_wise = QTable(names=('RAJ2000', 'DEJ2000', 'W1mag', 'W2mag', 'W3mag', 'W4mag'))
df_panstarrs = QTable(names=('RAJ2000', 'DEJ2000', 'gmag', 'rmag'))

for i in range(len(catalogs)):
    result_table = Vizier.query_region(coord, radius=rad, catalog=catalog[i])
    #result_table.pprint(show_unit=True, max_lines=100, max_width=1000)
    if i==0:
        df_gaia = result_table[0]['RA_ICRS','DE_ICRS', 'Gmag', 'e_Gmag']
    if i==1:
        df_2mass = result_table[0]['RAJ2000', 'DEJ2000', 'Jmag', 'Hmag', 'Kmag']
    if i==2:
        df_wise = result_table[0]['RAJ2000', 'DEJ2000', 'W1mag', 'W2mag', 'W3mag', 'W4mag']
    if i==3:
        df_panstarrs = result_table[0]['RAJ2000', 'DEJ2000', 'gmag', 'rmag', 'imag', 'zmag']

       
data_2mass = XMatch.query(cat1=df_gaia,
                     cat2=catalogs[1],
                     max_distance=3*u.arcsec,
                     colRA1='RA_ICRS', colDec1='DE_ICRS')

data_wise = XMatch.query(cat1=data_2mass,
                     cat2=catalogs[2],
                     max_distance=3*u.arcsec,
                     colRA1='RA_ICRS', colDec1='DE_ICRS')

data_panstarrs = XMatch.query(cat1=data_wise,
                     cat2=catalogs[3],
                     max_distance=3*u.arcsec,
                     colRA1='RA_ICRS', colDec1='DE_ICRS')
print(data_panstarrs.columns)
data.add_columns([data_panstarrs['RA_ICRS'], data_panstarrs['DE_ICRS'], data_panstarrs['Gmag'], data_panstarrs['e_Gmag'],
                  data_panstarrs['Jmag_2'], data_panstarrs['e_Jmag_2'], data_panstarrs['Hmag_2'], data_panstarrs['e_Hmag_2'], data_panstarrs['Kmag_2'], data_panstarrs['e_Kmag_2'],
                  data_panstarrs['W1mag'], data_panstarrs['e_W1mag'], data_panstarrs['W2mag'], data_panstarrs['e_W2mag'],
                  data_panstarrs['W3mag'], data_panstarrs['e_W3mag'], data_panstarrs['W4mag'], data_panstarrs['e_W4mag'],
                  data_panstarrs['gmag'], data_panstarrs['e_gmag'], data_panstarrs['rmag'], data_panstarrs['e_rmag'], data_panstarrs['imag'], 
                  data_panstarrs['e_imag'], data_panstarrs['zmag'], data_panstarrs['e_zmag'], data_panstarrs['ymag'], data_panstarrs['e_ymag']])
data=QTable(data)
gaia_flux = Column(data=3229*10**(-0.4*data['Gmag']), name='G_flux')
e_gaia_flux = Column(data=3229*0.4*10**(-0.4*(data['Gmag']-1))*data['e_Gmag'], name='e_G_flux')
gaia_wl = Column(data=[690]*len(data), name='G_wave_length')

J2mass_flux = Column(data=1594*10**(-0.4*data['Jmag_2']), name='J_flux')
e_J2mass_flux = Column(data=1594*0.4*10**(-0.4*(data['Jmag_2']-1))*data['e_Jmag_2'], name='e_J_flux')
J2mass_wl = Column(data=[1235]*len(data), name='J_wave_length')
H2mass_flux = Column(data=1024*10**(-0.4*data['Hmag_2']), name='H_flux')
e_H2mass_flux = Column(data=1024*0.4*10**(-0.4*(data['Hmag_2']-1))*data['e_Hmag_2'], name='e_H_flux')
H2mass_wl = Column(data=[1662]*len(data), name='H_wave_length')
K2mass_flux = Column(data=666.7*10**(-0.4*data['Kmag_2']), name='K_flux')
e_K2mass_flux = Column(data=666.7*0.4*10**(-0.4*(data['Kmag_2']-1))*data['e_Kmag_2'], name='e_K_flux')
K2mass_wl = Column(data=[2159]*len(data), name='K_wave_length')

W1_flux = Column(data=309.54*10**(-0.4*data['W1mag']), name='W1_flux')
e_W1_flux = Column(data=309.54*0.4*10**(-0.4*(data['W1mag']-1))*data['e_W1mag'], name='e_W1_flux')
W1_wl = Column(data=[3368]*len(data), name='W1_wave_length')
W2_flux = Column(data=171.79*10**(-0.4*data['W2mag']), name='W2_flux')
e_W2_flux = Column(data=171.79*0.4*10**(-0.4*(data['W2mag']-1))*data['e_W2mag'], name='e_W2_flux')
W2_wl = Column(data=[4618]*len(data), name='W2_wave_length')
W3_flux = Column(data=31.676*10**(-0.4*data['W3mag']), name='W3_flux')
e_W3_flux = Column(data=31.676*0.4*10**(-0.4*(data['W3mag']-1))*data['e_W3mag'], name='e_W3_flux')
W3_wl = Column(data=[12082]*len(data), name='W3_wave_length')
W4_flux = Column(data=8.3635*10**(-0.4*data['W4mag']), name='W4_flux')
e_W4_flux = Column(data=8.3635*0.4*10**(-0.4*(data['W4mag']-1))*data['W4mag'], name='e_W4_flux')
W4_wl = Column(data=[22194]*len(data), name='W4_wave_length')

gps_flux = Column(data=3964.03*10**(-0.4*data['gmag']), name = 'g_flux')
e_gps_flux = Column(data=3964.03*0.4*10**(-0.4*(data['gmag']-1))*data['e_gmag'], name = 'e_g_flux')
gps_wl = Column(data=[481]*len(data), name='g_wave_length')

rps_flux = Column(data=3173.02*10**(-0.4*data['rmag']), name = 'r_flux')
e_rps_flux = Column(data=3173.02*0.4*10**(-0.4*(data['rmag']-1))*data['e_rmag'], name = 'e_r_flux')
rps_wl = Column(data=[617]*len(data), name='r_wave_length')

ips_flux = Column(data=2575.36*10**(-0.4*data['imag']), name = 'i_flux')
e_ips_flux = Column(data=2575.36*0.4*10**(-0.4*(data['imag']-1))*data['e_imag'], name = 'e_i_flux')
ips_wl = Column(data=[752]*len(data), name='i_wave_length')

zps_flux = Column(data=2261.81*10**(-0.4*data['zmag']), name = 'z_flux')
e_zps_flux = Column(data=2261.81*0.4*10**(-0.4*(data['zmag']-1))*data['e_zmag'], name = 'e_z_flux')
zps_wl = Column(data=[866]*len(data), name='z_wave_length')

yps_flux = Column(data=2261.81*10**(-0.4*data['ymag']), name = 'y_flux')
e_yps_flux = Column(data=2180.40*10**(-0.4*(data['ymag']-1))*data['e_ymag'], name = 'e_y_flux')
yps_wl = Column(data=[962]*len(data), name='y_wave_length')

data.add_columns([gaia_flux, e_gaia_flux, gaia_wl,
                  J2mass_flux, e_J2mass_flux, J2mass_wl, H2mass_flux, e_H2mass_flux, H2mass_wl, K2mass_flux, e_K2mass_flux, K2mass_wl,
                  W1_flux, e_W1_flux, W1_wl, W2_flux, e_W2_flux, W2_wl, W3_flux, e_W3_flux, W3_wl, W4_flux, e_W4_flux, W4_wl,
                  gps_flux, e_gps_flux, gps_wl, rps_flux, e_rps_flux, rps_wl, ips_flux, e_ips_flux, ips_wl, zps_flux, e_zps_flux, zps_wl, yps_flux, e_yps_flux, yps_wl])
data.remove_columns(['Gmag', 'Jmag_2', 'Hmag_2', 'Kmag_2', 'W1mag', 'W2mag', 'W3mag', 'W4mag', 'gmag', 'rmag', 'imag', 'zmag', 'ymag',
                     'e_Gmag', 'e_Jmag_2', 'e_Hmag_2', 'e_Kmag_2', 'e_W1mag', 'e_W2mag', 'e_W3mag', 'e_W4mag', 'e_gmag', 'e_rmag', 'e_imag', 'e_zmag', 'e_ymag'])
"""
r['pmdec'] = r['pmdec'].apply(correct_pm)
plt.quiver(r["ra"].filled(0), r["dec"].filled(0), r["pmra"].filled(0), r["pmdec"].filled(0), color='0.4')
plt.show()
"""

<TableColumns names=('angDist','angDist_1','angDist_2','RA_ICRS','DE_ICRS','Gmag','e_Gmag','2MASS','RAJ2000_1','DEJ2000_1','errHalfMaj_1','errHalfMin_1','errPosAng_1','Jmag_1','Hmag_1','Kmag_1','e_Jmag_1','e_Hmag_1','e_Kmag_1','Qfl','Rfl','X','MeasureJD','AllWISE','RAJ2000_2','DEJ2000_2','eeMaj','eeMin','eePA','W1mag','W2mag','W3mag','W4mag','Jmag_2','Hmag_2','Kmag_2','e_W1mag','e_W2mag','e_W3mag','e_W4mag','e_Jmag_2','e_Hmag_2','e_Kmag_2','ID','ccf','ex','var','qph','pmRA','e_pmRA','pmDE','e_pmDE','d2M','objID','RAJ2000','DEJ2000','errHalfMaj_2','errHalfMin_2','errPosAng_2','f_objID','Qual','Epoch','Ns','Nd','gmag','e_gmag','gKmag','e_gKmag','gFlags','rmag','e_rmag','rKmag','e_rKmag','rFlags','imag','e_imag','iKmag','e_iKmag','iFlags','zmag','e_zmag','zKmag','e_zKmag','zFlags','ymag','e_ymag','yKmag','e_yKmag','yFlags')>


TypeError: Cannot convert a MaskedColumn with masked value to a Column

In [59]:
data

RA_ICRS,DE_ICRS,Gmag,e_Gmag,Jmag_2,e_Jmag_2,Hmag_2,e_Hmag_2,Kmag_2,e_Kmag_2,W1mag,e_W1mag,W2mag,e_W2mag,W3mag,e_W3mag,W4mag,e_W4mag,gmag,e_gmag,rmag,e_rmag,imag,e_imag,zmag,e_zmag,ymag,e_ymag
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
67.65393185732,64.76572693604,18.624233,0.003125,15.777,0.084,15.183,0.111,14.95,0.135,14.766,0.033,14.742,0.061,12.083,--,9.122,--,20.3149,0.0288,18.8344,0.0092,17.9763,0.0038,17.5237,0.0034,17.2128,0.0197
67.63863570041,64.76042696261,16.605988,0.002794,14.726,0.036,14.232,0.054,14.089,0.061,13.981,0.027,14.063,0.04,12.321,--,8.824,--,17.6146,0.0015,16.6575,0.0043,16.1582,0.0035,15.9031,0.0023,15.7287,0.0056
67.5994214376,64.76052089568,19.531567,0.00369,16.53,0.133,15.675,0.168,15.437,0.201,15.193,0.037,15.218,0.079,11.972,--,8.868,--,21.8132,0.0982,20.0055,0.0203,18.7804,0.0045,18.1511,0.0089,17.8124,0.0239
67.62341151427,64.76220729184,19.120481,0.01723,15.305,0.048,14.702,0.071,14.501,0.089,14.339,0.029,14.273,0.048,12.798,--,9.073,--,18.6489,0.0096,17.5686,0.0029,16.9991,0.0057,16.6444,0.0062,16.3907,0.007
67.63421636755,64.77217830975,18.319347,0.002983,14.805,0.042,14.192,0.061,14.082,0.066,13.67,0.026,13.753,0.036,12.497,0.469,9.067,--,--,--,18.4223,0.0227,17.7601,0.0028,17.3817,0.0059,17.1262,0.0233
67.63421636755,64.77217830975,18.319347,0.002983,14.805,0.042,14.192,0.061,14.082,0.066,13.67,0.026,13.753,0.036,12.497,0.469,9.067,--,18.043,0.0065,17.0126,0.0029,16.447,0.0007,16.1888,0.0053,15.9867,0.0074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67.60992404072,64.7969389951,17.032064,0.002801,15.27,0.051,14.869,0.075,14.562,0.089,14.66,0.032,14.643,0.052,12.671,--,9.139,--,17.8975,0.0085,17.0645,0.0043,16.6296,0.002,16.4058,0.0045,16.2403,0.0139
67.59771206665,64.79745183534,17.217918,0.002815,15.545,0.061,15.159,0.104,15.076,0.14,14.629,0.032,14.567,0.051,12.714,--,8.971,--,18.0329,0.0061,17.2437,0.0033,16.8286,0.0022,16.6162,0.0028,16.4879,0.0137


In [55]:
for i in range(len(data)):
    plt.loglog([data[i]["G_wave_length"], data[i]['J_wave_length'], data[i]['H_wave_length'], data[i]['K_wave_length'],
                 data[i]['W1_wave_length'], data[i]['W2_wave_length'], data[i]['W3_wave_length'], data[i]['W4_wave_length'],
                 data[i]['g_wave_length'], data[i]['r_wave_length'], data[i]['i_wave_length'], data[i]['z_wave_length'], data[i]['y_wave_length']],
                
                [data[i]['G_flux'], data[i]['J_flux'], data[i]['H_flux'], data[i]['K_flux'],
                 data[i]['W1_flux'], data[i]['W2_flux'], data[i]['W3_flux'], data[i]['W4_flux'],
                 data[i]['g_flux'], data[i]['r_flux'], data[i]['i_flux'], data[i]['z_flux'], data[i]['y_flux']],
                s=12, c='black')
    plt.xlabel("λ, nm")
    plt.ylabel("flux, Jy")
    plt.show()

KeyError: 'G_wave_length'